In [1]:
import spacy
from typing import Tuple
import nltk
import numpy as np
import pandas as pd
from functools import partial
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from scipy.stats import pearsonr

from zp_ihlt_project.config import TRAIN_DATA_DIR, TEST_DATA_DIR

In [2]:
all_train_dt = pd.read_csv("../data/processed/train_data_with_features.csv")
all_test_dt = pd.read_csv("../data/processed/test_data_with_features.csv")
feature_names = [col for col in all_train_dt.columns if col.startswith("score_")]

In [3]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

kbest = SelectKBest(mutual_info_regression, k=10000).fit(all_train_dt[feature_names], all_train_dt.gs)
selected_features = np.array(feature_names)[kbest.get_support()]

/Users/zachparent/src/MAI/ihlt/.venv-ihlt/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=10000 is greater than n_features=2080. All the features will be returned.
  warnings.warn(


In [4]:
train_datasets = all_train_dt.dataset.unique().tolist()
test_datasets = all_test_dt.dataset.unique().tolist()

In [5]:
results = []

for dataset in train_datasets:
    dataset_results = []
    dataset_results.append(dataset)
    dt = all_train_dt[all_train_dt.dataset == dataset]
    test_dt = all_test_dt[all_test_dt.dataset == dataset]

    X_train, X_val, y_train, y_val = train_test_split(dt[selected_features], dt.gs, test_size=0.2, random_state=42)
    X_test = test_dt[selected_features]
    y_test = test_dt.gs

    # Define parameter grid
    param_grid = {
        'C': [0.01, 0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto'],
        'epsilon': [0.1]
    }

    svr = SVR()

    # Perform grid search with 5-fold cross validation
    grid_search = GridSearchCV(
        svr, 
        param_grid,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )

    # Fit grid search
    grid_search.fit(X_train, y_train)

    # Print best parameters and score
    print("Best parameters:", grid_search.best_params_)
    print("Best RMSE:", np.sqrt(-grid_search.best_score_))

    best_mlp = grid_search.best_estimator_
    best_mlp.fit(X_train, y_train)
    preds = best_mlp.predict(X_train)
    dataset_results.append(pearsonr(y_train, preds)[0])

    preds = best_mlp.predict(X_val)
    dataset_results.append(pearsonr(y_val, preds)[0])

    preds = best_mlp.predict(X_test[selected_features])
    dataset_results.append(pearsonr(y_test, preds)[0])

    results.append(dataset_results)

results = pd.DataFrame(results, columns=["dataset", "train_pearson", "val_pearson", "test_pearson"])
results

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters: {'C': 0.01, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Best RMSE: 0.7126624251054213
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters: {'C': 10, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Best RMSE: 0.979758314136345
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters: {'C': 10, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Best RMSE: 0.5069494395014974


,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.698919,0.712740,0.631109
1,MSRvid,0.879382,0.854750,0.820580
2,SMTeuroparl,0.855186,0.686114,0.355875


In [6]:
dt = all_train_dt

X_train, X_val, y_train, y_val = train_test_split(dt[selected_features], dt.gs, test_size=0.2, random_state=42)
X_test = all_test_dt[selected_features]
y_test = all_test_dt.gs

results = []

# Define parameter grid
param_grid = {
    'C': [1, 5, 10, 50, 100],
    'kernel': ['rbf', 'poly'],
    'gamma': ['scale'],
    'epsilon': [0.1, 0.2, 0.3]
}

svr = SVR()

# Perform grid search with 5-fold cross validation
grid_search = GridSearchCV(
    svr, 
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# Fit grid search
grid_search.fit(X_train, y_train)

# Print best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))

best_mlp = grid_search.best_estimator_
preds = best_mlp.predict(X_train)
results.append(pearsonr(y_train, preds)[0])

preds = best_mlp.predict(X_val)
results.append(pearsonr(y_val, preds)[0])

preds = best_mlp.predict(X_test)
results.append(pearsonr(y_test, preds)[0])

results = pd.DataFrame([results], columns=["train_pearson", "val_pearson", "test_pearson"], index=["all"])
results

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters: {'C': 10, 'epsilon': 0.3, 'gamma': 'scale', 'kernel': 'rbf'}
Best RMSE: 0.7580333798614225


,train_pearson,val_pearson,test_pearson
all,0.897373,0.837297,0.723818


In [7]:
best_mlp.fit(dt[selected_features], dt.gs)

SVR(C=10, epsilon=0.3)

In [8]:
train_results = []
datasets = train_datasets
for dataset in datasets:
    dataset_results = []
    dataset_results.append(dataset)
    dt = all_train_dt[all_train_dt.dataset == dataset]
    test_dt = all_test_dt[all_test_dt.dataset == dataset]

    X_train, X_val, y_train, y_val = train_test_split(dt[selected_features], dt.gs, test_size=0.2, random_state=42)
    X_test = test_dt[selected_features]
    y_test = test_dt.gs

    preds = best_mlp.predict(X_train)
    dataset_results.append(pearsonr(y_train, preds)[0])

    preds = best_mlp.predict(X_val)
    dataset_results.append(pearsonr(y_val, preds)[0])

    preds = best_mlp.predict(X_test[selected_features])
    dataset_results.append(pearsonr(y_test, preds)[0])

    train_results.append(dataset_results)

train_results = pd.DataFrame(train_results, columns=["dataset", "train_pearson", "val_pearson", "test_pearson"])
train_results

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.747261,0.754107,0.631662
1,MSRvid,0.859978,0.907385,0.825297
2,SMTeuroparl,0.763727,0.726645,0.511699


In [9]:
test_results = []
datasets = test_datasets
for dataset in datasets:
    dataset_results = []
    dataset_results.append(dataset)
    test_dt = all_test_dt[all_test_dt.dataset == dataset]

    X_test = test_dt[selected_features]
    y_test = test_dt.gs

    preds = best_mlp.predict(X_test[selected_features])
    dataset_results.append(pearsonr(y_test, preds)[0])

    test_results.append(dataset_results)

preds = best_mlp.predict(all_test_dt[selected_features])
test_results.append(['all', pearsonr(all_test_dt.gs, preds)[0]])
test_results = pd.DataFrame(test_results, columns=["dataset", "test_pearson"])
test_results

,dataset,test_pearson
0,MSRpar,0.631662
1,MSRvid,0.825297
2,SMTeuroparl,0.511699
3,OnWN,0.643387
4,SMTnews,0.462615
5,all,0.729040


In [10]:
results_to_beat = pd.DataFrame(np.array([[.683, .873, .528, .664, .493, 0.823]]).T, index=[*test_datasets, 'all'], columns=["pearson_to_beat"])
results_to_beat

,pearson_to_beat
MSRpar,0.683
MSRvid,0.873
SMTeuroparl,0.528
OnWN,0.664
SMTnews,0.493
all,0.823
